## STEPS
### 1. read csv and create a list of all pmid's and pmcid's (IGNORE THIS PART FOR NOW // still in progress)

In [ ]:
import csv

In [34]:
def extract_values_from_csv(csv_file):
    column1_values = []
    column2_values = []

    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row

        for row in csv_reader:
            # Split the row by commas and convert non-empty values to integers
            values = [int(x.strip()) for x in row[0].split(',') if x.strip()]

            # Assuming the first value is from the first column and the second value is from the second column
            if len(values) >= 1:
                column1_values.extend(values)  # Use extend to add multiple values
            if len(values) >= 2:
                column2_values.extend(values)  # Use extend to add multiple values

    return column1_values, column2_values

In [42]:
# Usage example:
csv_file = '/Users/zeynepkorkmaz/Desktop/Master 1/Systems Biomedicine/NLP in biomedical research/pmid_pmcid_TP.csv'  # Replace with the path to your CSV file
column1_values, column2_values = extract_values_from_csv(csv_file)
#print("Values from column 1:", column1_values)
#print("Values from column 2:", column2_values)


In [43]:
len(column2_values)

5071

In [44]:
len(column1_values)

5076

#### Both columns should be of same length --> this is due to the Nan values --> Will fix this part once we take care of that issue

## 2. Change the script such that the found XML files are saved in a directory seperately

In [23]:
from lxml import etree as ET
import os
import shutil

In [24]:

def extract_and_save_xml_files_lxml(input_list_pmids, input_list_pmcids, input_dir, output_dir):
    previously_found_pmids = []
    previously_found_pmcids = []

    # Create the output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for subdir, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.xml'):
                xml_file_path = os.path.join(subdir, file)
                try:
                    parser = ET.XMLParser(recover=True, no_network=False)
                    tree = ET.parse(xml_file_path, parser)
                    root = tree.getroot()

                    for element in root.xpath('.//article-id'):
                        pub_id_type = element.get('pub-id-type')
                        text = element.text

                        if ((pub_id_type == 'pmid' and text in input_list_pmids and text not in previously_found_pmids) or 
                            (pub_id_type == 'pmc' and text in input_list_pmcids and text not in previously_found_pmcids)):
                            
                            # Save the XML file in the output directory
                            output_file_path = os.path.join(output_dir, file)
                            tree.write(output_file_path, encoding='utf-8', xml_declaration=True, pretty_print=True)

                            # Keep track of added pmids and pmcids
                            if pub_id_type == 'pmid':
                                previously_found_pmids.append(text)
                            if pub_id_type == 'pmc':
                                previously_found_pmcids.append(text)
                            
                            break  # Break the loop once a valid 'article-id' is found
                except ET.ParseError as e:
                    print(f"Skipping file {xml_file_path} due to parse error: {e}")


In [47]:
# This is just for testing
# Once the issue in the first step is handled, this part will be modified
test_pmids = ["34035045", "34035044", "33888511", "12929205"] # first three IDs version 1.3 and last one 1.0
test_pmcids = ["PMC8494224", "PMC8494231", "PMC8494234", "PMC176545"] # first three IDs version 1.3 and last one 1.0

In [45]:
# Define input and output directory
input_dir = '/Users/zeynepkorkmaz/Downloads/All_Articles/PMC000xxxxxx'
output_dir = "/Users/zeynepkorkmaz/Downloads/separate_XML"

In [28]:
extract_and_save_xml_files_lxml(test_pmids,test_pmcids,input_dir,output_dir )